[View in Colaboratory](https://colab.research.google.com/github/Robert-Alonso/Kaggle-Titanic/blob/master/DisRegGen_CIFAR10_Final.ipynb)

# <center>Discriminative Regularize Generative Model for CIFAR10 </center>

## Load Data

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
# !pip3 install --upgrade torch torchvision

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable

import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from IPython.display import Image
# from google.colab import files

#Set random seed 
torch.manual_seed(512)

In [3]:
class preTrainedModel(nn.Module):
  
    def __init__(self):
      
      super(preTrainedModel,self).__init__()
      
      vgg_model = torchvision.models.vgg16(pretrained=True)		
      self.Conv1 = nn.Sequential(*list(vgg_model.features.children())[0:4])
      self.Conv2 = nn.Sequential(*list(vgg_model.features.children())[4:9]) 
      #self.Conv3 = nn.Sequential(*list(vgg_model.features.children())[9:16])
      #self.upSample1 = nn.Upsample(scale_factor=2)
      #self.upSample2 = nn.Upsample(scale_factor=4)
      
      for param in self.parameters():
        param.requires_grad = False

    def forward(self,x):
      out1 = self.Conv1(x)
      out2 = self.Conv2(out1)
      #out3 = self.Conv3(out2)
      ###### up sampling to create output with the same size
      #out2 = self.upSample1(out2)
      #out3 = self.upSample2(out3)
      #concat_features = torch.cat([out1, out2, out3], 1)
      return (out1,out2)



In [4]:
class Vgg16(nn.Module):
  
    def __init__(self):
        super(Vgg16, self).__init__()
        features = torchvision.models.vgg16(pretrained=True).features
        self.to_relu_1_2 = nn.Sequential() 
        self.to_relu_2_2 = nn.Sequential() 
        self.to_relu_3_3 = nn.Sequential()
        self.to_relu_4_3 = nn.Sequential()

        for x in range(4):
            self.to_relu_1_2.add_module(str(x), features[x])
        for x in range(4, 9):
            self.to_relu_2_2.add_module(str(x), features[x])
        for x in range(9, 16):
            self.to_relu_3_3.add_module(str(x), features[x])
        for x in range(16, 23):
            self.to_relu_4_3.add_module(str(x), features[x])
        
        # don't need the gradients, just want the features
        for param in self.parameters():
            param.requires_grad = False

    def forward(self, x):
        h = self.to_relu_1_2(x)
        h_relu_1_2 = h
        h = self.to_relu_2_2(h)
        h_relu_2_2 = h
        h = self.to_relu_3_3(h)
        h_relu_3_3 = h
        h = self.to_relu_4_3(h)
        h_relu_4_3 = h
        out = (h_relu_1_2, h_relu_2_2, h_relu_3_3, h_relu_4_3)
        return out

In [5]:
#Load model 
vgg16 = preTrainedModel().eval().cuda()
#vgg16 = Vgg16().eval().cuda()

In [6]:
#Get the CIFAR10 train images 
cifar = datasets.CIFAR10('./data/cifar/', train = True, download = True)

# Organize training data in batches, 
# normalize them to have values between [-1, 1] (?)

train_images = torch.utils.data.DataLoader ( datasets.CIFAR10('./data/cifar/', train = True, download=False,
                               transform=transforms.Compose([
                               #transforms.Resize(64), 
                               #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                               transforms.ToTensor(),])) , 
                               batch_size = 64, shuffle = True)

Files already downloaded and verified


In [7]:
upsampling = nn.Upsample(size=256)
for batch_idx, (data,_) in enumerate(train_images):    
    out = vgg16(upsampling(data.cuda()))
    
    print('Data size: ', data.size())
    print('Output 1 size: ', out[0].size())
    print('Output 2 size: ', out[1].size())
    #print('Output 3 size: ', out[2].size())
    #print('Output 4 size: ', out[3].size())
    
    break

Data size:  torch.Size([64, 3, 32, 32])
Output 1 size:  torch.Size([64, 64, 256, 256])
Output 2 size:  torch.Size([64, 128, 128, 128])


In [8]:
upsampling = nn.Upsample(size=256)
for batch_idx, (data,_) in enumerate(train_images):    
    out = vgg16(upsampling(data.cuda()))
    
    print('Data size: ', data.size())
    print('Output 1 size: ', out[0].size())
    print('Output 2 size: ', out[1].size())
    #print('Output 3 size: ', out[2].size())
    #print('Output 4 size: ', out[3].size())
    
    break

Data size:  torch.Size([64, 3, 32, 32])
Output 1 size:  torch.Size([64, 64, 256, 256])
Output 2 size:  torch.Size([64, 128, 128, 128])


## Model

We will use the arquitecture suggested by [Radford et al](https://arxiv.org/abs/1511.06434) for both the encoder and decoder. With convolutional layers in the encoder and fractionally-strided  convolutions  in  the  decoder.   In  each convolutional layer in the encoder we double the number of filters present in the previous layer and use a convolutional stride of 2.  In each convolutional layer in the decoder we use a fractional stride of 2 and halve the number of filters on each layer.

In [9]:
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable

class VAE( nn.Module ):

    def __init__ ( self, image_size ,  hidden_dim , encoding_dim ):
        
        super( VAE, self ).__init__()
        
        self.encoding_dim = encoding_dim
        self.image_size = image_size
        self.hidden_dim = hidden_dim 
        
        # Decoder - Fractional strided convolutional layers
        self.decoder  = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 4, 1, 0, bias = False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias = False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 32, 4, 2, 1, bias = False),
            nn.BatchNorm2d(32),
            nn.ReLU(True),
            nn.ConvTranspose2d(32, 3, 4, 2, 1, bias = False),
            nn.Sigmoid() # nn.Tanh()  
        )
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 4, 2, 1, bias = False),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(32, 64, 4, 2, 1, bias = False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(64, 128, 4, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(128, 256, 4, 2, 0, bias = False),
            nn.Sigmoid()
        )
        
        # Fully-connected layers
        self.fc1 = nn.Linear(256, self.hidden_dim)
        self.fc21 = nn.Linear(self.hidden_dim, self.encoding_dim)
        self.fc22 = nn.Linear(self.hidden_dim, self.encoding_dim)
        self.fc3 = nn.Linear(self.encoding_dim, self.hidden_dim)
        self.fc4 = nn.Linear(self.hidden_dim, 256)
    
    def decode (self, z):
        h3 = F.relu(self.fc3(z))
        h4 = F.sigmoid(self.fc4(h3))
        return self.decoder( h4.view(z.size(0),-1,1,1) ) 

        
    def forward(self, x):
        
        # Encode 
        encoded = F.relu(self.fc1( self.encoder(x).view(x.size(0), -1) ) )
        
        #Obtain mu and logvar
        mu = self.fc21( encoded )
        logvar = self.fc22 ( encoded )
        
        #Reparametrization trick
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        z = eps.mul(std).add_(mu)
        
        # Decode 
        decoded = self.decode(z)

        # return decoded, mu, logvar
        return decoded, mu , logvar


upsampling = nn.Upsample(size=256)
sigmoid = nn.Sigmoid()

# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
  
  # VAE LOSS
    BCE = F.binary_cross_entropy(recon_x, x, size_average=False)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
  
  # PERCEPTUAL LOSS
    recon_x_vgg16 = vgg16 ( upsampling( recon_x ) )
    x_vgg16 = vgg16 ( upsampling( x ) )
    
    L0 = F.mse_loss( recon_x_vgg16[0] , x_vgg16[0], size_average=False)
    L1 = F.mse_loss( recon_x_vgg16[1] , x_vgg16[1], size_average=False)
    #L2 = F.mse_loss( recon_x_vgg16[2] , x_vgg16[2], size_average=False)
    #L3 = F.mse_loss( recon_x_vgg16[3] , x_vgg16[3], size_average=False)
    #print('L0: ',L0.data)
    #print('L1:' ,L1.data)
    #print('L2: ',L2.data)
    #print('L3: ', L3.data)
    #print(BCE.data)
    #print(KLD.data)
    
    del recon_x_vgg16 
    del x_vgg16
    
    return BCE + KLD + L0 + L1

In [10]:
#Define model
model = VAE( 32, 100, 20 ).cuda()
#model.load_state_dict(torch.load('save_checkpoint_epoch_9.pth'))
  
  
optimizer = optim.Adam(model.parameters(), lr=1e-3)

#Train model
def train(epoch):
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_images):
        data = Variable(data).cuda()
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 50 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_images.dataset),
                100. * batch_idx / len(train_images),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_images.dataset)))

In [11]:
# Titan XP (bs 64, 9 epochs) 
num_epochs = 10
for epoch in range(num_epochs):
    train(epoch)    

Train Epoch: 1 [0/50000 (0%)]	Loss: 1421915.500000
Train Epoch: 1 [3200/50000 (6%)]	Loss: 1367177.750000
Train Epoch: 1 [6400/50000 (13%)]	Loss: 1296118.375000
Train Epoch: 1 [9600/50000 (19%)]	Loss: 1245974.500000
Train Epoch: 1 [12800/50000 (26%)]	Loss: 1255630.125000
Train Epoch: 1 [16000/50000 (32%)]	Loss: 1204967.750000
Train Epoch: 1 [19200/50000 (38%)]	Loss: 1331030.125000
Train Epoch: 1 [22400/50000 (45%)]	Loss: 1257345.750000
Train Epoch: 1 [25600/50000 (51%)]	Loss: 1202275.500000
Train Epoch: 1 [28800/50000 (58%)]	Loss: 1213967.500000
Train Epoch: 1 [32000/50000 (64%)]	Loss: 1288281.125000
Train Epoch: 1 [35200/50000 (70%)]	Loss: 1248967.750000
Train Epoch: 1 [38400/50000 (77%)]	Loss: 1274365.500000
Train Epoch: 1 [41600/50000 (83%)]	Loss: 1222662.000000
Train Epoch: 1 [44800/50000 (90%)]	Loss: 1263295.375000
Train Epoch: 1 [48000/50000 (96%)]	Loss: 1189357.250000
====> Epoch: 1 Average loss: 1244314.8620
Train Epoch: 2 [0/50000 (0%)]	Loss: 1235518.250000
Train Epoch: 2 [3200

In [13]:
# Titan XP (bs 4, 4 epochs) 
num_epochs = 5
for epoch in range(1,num_epochs):
    train(epoch)    

Train Epoch: 1 [0/50000 (0%)]	Loss: 1629590.250000
Train Epoch: 1 [200/50000 (0%)]	Loss: 1287580.875000
Train Epoch: 1 [400/50000 (1%)]	Loss: 1012471.000000
Train Epoch: 1 [600/50000 (1%)]	Loss: 1147207.500000
Train Epoch: 1 [800/50000 (2%)]	Loss: 1462456.000000
Train Epoch: 1 [1000/50000 (2%)]	Loss: 1260769.750000
Train Epoch: 1 [1200/50000 (2%)]	Loss: 1381871.250000
Train Epoch: 1 [1400/50000 (3%)]	Loss: 1221112.500000
Train Epoch: 1 [1600/50000 (3%)]	Loss: 1815478.250000
Train Epoch: 1 [1800/50000 (4%)]	Loss: 1444160.250000
Train Epoch: 1 [2000/50000 (4%)]	Loss: 1087770.875000
Train Epoch: 1 [2200/50000 (4%)]	Loss: 1309435.625000
Train Epoch: 1 [2400/50000 (5%)]	Loss: 1534612.625000
Train Epoch: 1 [2600/50000 (5%)]	Loss: 1325607.500000
Train Epoch: 1 [2800/50000 (6%)]	Loss: 1389109.750000
Train Epoch: 1 [3000/50000 (6%)]	Loss: 1298786.000000
Train Epoch: 1 [3200/50000 (6%)]	Loss: 1050982.250000
Train Epoch: 1 [3400/50000 (7%)]	Loss: 1183835.250000
Train Epoch: 1 [3600/50000 (7%)]	Lo

In [21]:
# Google Collaboratory (original bs 4, 4 epochs)
num_epochs = 5
for epoch in range(1,num_epochs):
    train(epoch)    

Train Epoch: 1 [0/50000 (0%)]	Loss: 116313.093750
Train Epoch: 1 [200/50000 (0%)]	Loss: 177329.093750
Train Epoch: 1 [400/50000 (1%)]	Loss: 90363.796875
Train Epoch: 1 [600/50000 (1%)]	Loss: 88329.031250
Train Epoch: 1 [800/50000 (2%)]	Loss: 111293.375000
Train Epoch: 1 [1000/50000 (2%)]	Loss: 76695.054688
Train Epoch: 1 [1200/50000 (2%)]	Loss: 139406.093750
Train Epoch: 1 [1400/50000 (3%)]	Loss: 85086.632812
Train Epoch: 1 [1600/50000 (3%)]	Loss: 55279.351562
Train Epoch: 1 [1800/50000 (4%)]	Loss: 86910.640625
Train Epoch: 1 [2000/50000 (4%)]	Loss: 65911.195312
Train Epoch: 1 [2200/50000 (4%)]	Loss: 133015.531250
Train Epoch: 1 [2400/50000 (5%)]	Loss: 77796.062500
Train Epoch: 1 [2600/50000 (5%)]	Loss: 92703.843750
Train Epoch: 1 [2800/50000 (6%)]	Loss: 97212.656250
Train Epoch: 1 [3000/50000 (6%)]	Loss: 122148.476562
Train Epoch: 1 [3200/50000 (6%)]	Loss: 133246.968750
Train Epoch: 1 [3400/50000 (7%)]	Loss: 73505.500000
Train Epoch: 1 [3600/50000 (7%)]	Loss: 64181.964844
Train Epoch:

KeyboardInterrupt: ignored

In [13]:
with torch.no_grad():
        sample = torch.randn(64, 20).cuda()
        sample = model.decode(sample)
        torchvision.utils.save_image(sample.view(64, 3, 32, 32),'../results/sample_' + str(epoch) + '.png')
#         files.download('sample_' + str(epoch) + '.png')

In [14]:
torch.save(model.cpu().state_dict(), "../models/save_checkpoint_epoch_"+str(epoch)+".pth")
# files.download("save_checkpoint_epoch_"+str(epoch)+".pth")

In [22]:
!pip install Pillow==4.0.0
!pip install PIL
!pip install image

sample = torch.randn(64, 20).cuda()
sample = model.decode(sample)
%matplotlib inline
torchvision.utils.save_image(sample.view(64, 3, 32, 32),'sample_' + str(epoch) + '.png')
files.download('sample_' + str(epoch) + '.png')

  Using cached https://files.pythonhosted.org/packages/37/e8/b3fbf87b0188d22246678f8cd61e23e31caa1769ebc06f1664e2e5fe8a17/Pillow-4.0.0-cp36-cp36m-manylinux1_x86_64.whl
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
  Found existing installation: Pillow 5.1.0
    Uninstalling Pillow-5.1.0:
      Successfully uninstalled Pillow-5.1.0
  Could not find a version that satisfies the requirement PIL (from versions: )
No matching distribution found for PIL
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
